In [1]:
from extractor import get_text
from scan import get_files
import pandas as pd
import os
import json
from prompt import *
from pathlib import Path
from tqdm.auto import tqdm
import requests

In [2]:
PATH = r"C:\Users\alika\OneDrive\Masaüstü\AuditAssistant\Raporlar"

In [26]:
directory = Path(PATH)
rows = []
for f in directory.iterdir():
    if f.is_file():
        rows.append({
        "Full Path": str(f.resolve()),
        "File Name": f.stem,
        "Extension": f.suffix.lstrip('.')
        })

df = pd.DataFrame(rows).head(10)

In [27]:

from ollama import chat, generate
model = 'qwen3:8b'
question = 'Is there any personal information in the document ?'
def ask(text, question, model=model):
    resp = requests.post(
        "http://localhost:11434/api/chat",
        json={
            "model": model,
            "messages":[{'role': 'system', 'content': sp, }, 
                        {'role': 'user', 'content': up.format(text, question),}],
            "stream": False,
            'format':"json",
            "options": {"temperature": 0.0,
                        "top_p": 0.9,
                        #"num_predict": 256,   # max tokens to generate
                        #"stop": ["</end>"],   # optional stop tokens
            },
        },
    )    
    return resp


In [33]:
js = list()
for idx, path in tqdm(df.iterrows()):
    text = get_text(path['Full Path'])
    response = ask(text, question)
    js.append((text, response.text))
    

0it [00:00, ?it/s]

In [68]:
xx = js[0][1]

In [72]:
json.loads(xx)['total_duration']

31076783100

31.0767831

'00:31'

In [36]:
df_out['content'] = df_out['out_js'].apply(lambda row: row['message']['content'])

In [ ]:
df_out['content'] = df_out['content'].apply(lambda row: json.loads(row))

0    {'type': 'Report', 'language': 'Turkish', 'ans...
1    {'type': 'Report', 'language': 'Turkish', 'ans...
2    {'type': 'Report', 'language': 'Turkish', 'ans...
3    {'type': 'Report', 'language': 'Turkish', 'ans...
4    {'type': 'Report', 'language': 'Turkish', 'ans...
5    {'type': 'Report', 'language': 'Turkish', 'ans...
6    {'type': 'Report', 'language': 'Turkish', 'ans...
7    {'type': 'Report', 'language': 'Turkish', 'ans...
8    {'type': 'Report', 'language': 'Turkish', 'ans...
9    {'type': 'report', 'language': 'tr', 'answer':...
Name: content, dtype: object